# Quadratic Constraints

This example demonstrates how to create and solve models with quadratic constraints (QCP/QCQP). A quadratic constraint has the form:

$$ x^T Q x + a^T x \leq b $$

Quadratic constraints work seamlessly with coordinates, just like linear constraints.

In [ ]:
import pandas as pd

import linopy

## Problem Setup

We model a resource allocation problem over multiple time periods. For each time $t$, we want to maximize $x_t + 2y_t$ subject to:
- A budget constraint: $x_t + y_t \leq b_t$
- A risk constraint (quadratic): $x_t^2 + y_t^2 \leq r_t$

where the budget $b_t$ and risk limit $r_t$ vary over time.

In [ ]:
m = linopy.Model()

time = pd.Index(range(5), name="time")

x = m.add_variables(lower=0, coords=[time], name="x")
y = m.add_variables(lower=0, coords=[time], name="y")

## Adding Constraints

Linear constraints work as usual. We use a `pd.Series` for the time-varying budget:

In [ ]:
budget = pd.Series([8, 9, 10, 11, 12], index=time)

m.add_constraints(x + y <= budget, name="budget")

Quadratic constraints use `add_quadratic_constraints`. The risk limits also vary over time:

In [ ]:
risk_limit = pd.Series([20, 25, 30, 35, 40], index=time)

m.add_quadratic_constraints(x**2 + y**2, "<=", risk_limit, name="risk")

## Objective and Solve

We maximize the sum over all time periods:

In [ ]:
m.add_objective((x + 2 * y).sum(), sense="max")
m

To retrieve dual values for quadratic constraints with Gurobi, set `QCPDual=1`:

In [ ]:
m.solve(solver_name="gurobi", QCPDual=1)

## Results

In [ ]:
m.solution.to_dataframe().plot(kind="bar", ylabel="Optimal Value", rot=0);

## Inspecting Quadratic Constraints

Quadratic constraints are stored in `m.quadratic_constraints`:

In [ ]:
m.quadratic_constraints

In [ ]:
m.quadratic_constraints["risk"]

Dual values are available via the `.dual` property:

In [ ]:
m.quadratic_constraints["risk"].dual

## Bilinear Terms

Quadratic constraints can also include cross-product terms like $xy$. Note that bilinear constraints are nonconvex and require a solver that supports them (e.g., Gurobi):

In [ ]:
m2 = linopy.Model()

x = m2.add_variables(lower=0, name="x")
y = m2.add_variables(lower=0, name="y")

# Bilinear constraint: xy <= 4
m2.add_quadratic_constraints(x * y, "<=", 4, name="bilinear")

m2.add_objective(x + y, sense="max")
m2.add_constraints(x <= 5)
m2.add_constraints(y <= 5)

m2.solve(solver_name="gurobi")

print(f"x = {float(x.solution):.2f}, y = {float(y.solution):.2f}")

## Mixed Linear and Quadratic Terms

Quadratic constraints can combine both quadratic and linear terms in the same constraint. For example, $x^2 + 2x + y \leq 10$:

In [ ]:
m3 = linopy.Model()

x = m3.add_variables(lower=0, name="x")
y = m3.add_variables(lower=0, name="y")

# Mixed constraint: x² + 2x + y <= 10
m3.add_quadratic_constraints(x**2 + 2 * x + y, "<=", 10, name="mixed")

m3.add_objective(x + y, sense="max")

m3.solve(solver_name="gurobi")

print(f"x = {float(x.solution):.2f}, y = {float(y.solution):.2f}")

## Equality Constraints

Quadratic equality constraints use `"=="`. This example constrains a point to lie exactly on a circle:

In [ ]:
m4 = linopy.Model()

x = m4.add_variables(lower=-5, upper=5, name="x")
y = m4.add_variables(lower=-5, upper=5, name="y")

# Point must lie on circle of radius 2
m4.add_quadratic_constraints(x**2 + y**2, "==", 4, name="circle")

# Maximize x + y (find point on circle furthest in direction (1,1))
m4.add_objective(x + y, sense="max")

m4.solve(solver_name="gurobi")

print(f"x = {float(x.solution):.4f}, y = {float(y.solution):.4f}")
print(f"x² + y² = {float(x.solution) ** 2 + float(y.solution) ** 2:.4f}")

## Convexity Considerations

Quadratic constraints have important convexity properties that affect which solvers can handle them:

- **Convex** (most solvers): $x^T Q x + a^T x \leq b$ where $Q$ is positive semidefinite (e.g., sum of squares like $x^2 + y^2$)
- **Nonconvex** (requires specialized solvers like Gurobi):
  - $x^T Q x + a^T x \geq b$ (greater-than with positive semidefinite Q)
  - $x^T Q x + a^T x = b$ (equality constraints)
  - Bilinear terms like $xy$

Convex quadratic constraints define a convex feasible region (like the interior of an ellipse), while nonconvex constraints can create disconnected or non-convex regions. Solvers like Gurobi use spatial branch-and-bound to handle nonconvex cases.